<a href="https://colab.research.google.com/github/somefork2/skillfactory_rds/blob/master/sf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pandas import Series
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import auc, roc_auc_score, roc_curve
import math
import datetime as dt
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')
data_path = 'drive/My Drive/sf_data/'
train = pd.read_csv(data_path+'train.csv')
train['label'] = 'train'
test = pd.read_csv(data_path+'test.csv')
test['label'] = 'test'
df = pd.concat([train,test])
df.education = df.education.apply(lambda x: 'SCH' if x is np.nan else x)
cat_cols = ['education','sex','car_type','home_address','foreign_passport']

if df.app_date.dtype != 'datetime64[ns]':#чтобы не вызывало ошибку при повторном вызове
    df.app_date = pd.to_datetime(df.app_date, format='%d%b%Y')
    
if len(df.columns) == len(train.columns): 
    onehot = pd.get_dummies(df[cat_cols])
    df = pd.concat([df,onehot],axis=1)
    print(1)


df.income = df.income.apply(np.log)

Mounted at /content/drive
1


In [ ]:
def throw_away_outliers(df):
  _df = df.copy()
  for column in df.columns:
    Q1 = _df[column].quantile(.25)
    Q3 = _df[column].quantile(.75)
    IQR = (Q3-Q1)*1.5
    _df = _df[(_df[column] > Q1-1.5*IQR ) | (_df[column] < Q1+1.5*IQR)]
    return _df

X = df[df.label=='train'][['age',
       'decline_app_cnt', 'good_work', 'score_bki', 'bki_request_cnt',
       'region_rating', 'income',
       'first_time',
       'education_ACD', 'education_GRD', 'education_PGR', 'education_SCH',
       'education_UGR', 'sex_F', 'sex_M', 'car_type_N', 'car_type_Y',
       'foreign_passport_N', 'foreign_passport_Y']]
X = throw_away_outliers(X)
X = StandardScaler().fit_transform(X)
y = df[df.label=='train'].default
model = LogisticRegression(random_state=0,class_weight='balanced',penalty='l2', C=7)
model.fit(X,y)
probs = model.predict_proba(X)
y_pred = model.predict(X)

proba = pd.DataFrame(probs[:,:1])

print(roc_auc_score(y,proba))
print(confusion_matrix(y_true=y,y_pred=y_pred))

0.2905276400230401
[[41524 22903]
 [ 3207  6165]]


In [ ]:
df

,client_id,app_date,education,sex,age,car,car_type,decline_app_cnt,good_work,score_bki,bki_request_cnt,region_rating,home_address,work_address,income,sna,first_time,foreign_passport,default,label,home_address,education_ACD,education_GRD,education_PGR,education_SCH,education_UGR,sex_F,sex_M,car_type_N,car_type_Y,foreign_passport_N,foreign_passport_Y
0,25905,2014-02-01,SCH,M,62,Y,Y,0,0,-2.008753,1,50,1,2,9.798127,4,1,N,0.0,train,1,0,0,0,1,0,0,1,0,1,1,0
1,63161,2014-03-12,SCH,F,59,N,N,0,0,-1.532276,3,50,2,3,9.852194,4,1,N,0.0,train,2,0,0,0,1,0,1,0,1,0,1,0
2,25887,2014-02-01,SCH,M,25,Y,N,2,0,-1.408142,1,80,1,2,10.308953,1,4,Y,0.0,train,1,0,0,0,1,0,0,1,1,0,0,1
3,16222,2014-01-23,SCH,F,53,N,N,0,0,-2.057471,2,50,2,3,9.210340,1,3,N,0.0,train,2,0,0,0,1,0,1,0,1,0,1,0
4,101655,2014-04-18,GRD,M,48,N,N,0,1,-1.244723,1,60,2,3,10.308953,1,4,Y,0.0,train,2,0,1,0,0,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36344,83775,2014-03-31,SCH,F,37,N,N,1,0,-1.744976,3,50,2,3,9.615805,4,1,N,NaN,test,2,0,0,0,1,0,1,0,1,0,1,0
36345,106254,2014-04-25,GRD,F,64,Y,Y,0,0,-2.293781,3,60,1,2,12.206073,1,4,N,NaN,test,1,0,1,0,0,0,1,0,0,1,1,0
36346,81852,2014-03-30,GRD,M,31,N,N,2,0,-0.940752,1,50,1,2,11.002100,4,2,N,NaN,test,1,0,1,0,0,0,0,1,1,0,1,0
36347,1971,2014-01-07,UGR,F,27,N,N,1,0,-1.242392,2,80,2,3,10.308953,1,1,N,NaN,test,2,0,0,0,0,1,1,0,1,0,1,0


In [ ]:
from sklearn.model_selection import GridSearchCV

C = np.logspace(-1, 3, 10)
iter_ = 1000
epsilon_stop = 1e-3
 
hyperparameters = [
    {'penalty': ['l1'], 
     'C': C,
     'solver': ['liblinear', 'saga'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['l2'], 
     'C': C,
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['none'], 
     'C': C,
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['elasticnet'], 
     'C': C,
     'solver': ['saga'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
]

X = df[df.label=='train'][['age',
       'decline_app_cnt', 'good_work', 'score_bki', 'bki_request_cnt',
       'region_rating', 'income',
       'first_time',
       'education_ACD', 'education_GRD', 'education_PGR', 'education_SCH',
       'education_UGR', 'sex_F', 'sex_M', 'car_type_N', 'car_type_Y',
       'foreign_passport_N', 'foreign_passport_Y']]


probs = model.predict_proba(X)
y_pred = model.predict(X)
ids = pd.DataFrame(df[df.label=='train'].client_id.values)
ids.columns = ['client_id']
proba = pd.DataFrame(probs[:,:1])
res = pd.concat([ids,proba],axis=1)
print(roc_auc_score(y,proba))
print(confusion_matrix(y_true=y,y_pred=y_pred))


gridsearch = GridSearchCV(model, hyperparameters, scoring='roc_auc', n_jobs=-1, cv=5,verbose=1)
gridsearch.fit(X, y)
model_best_params = gridsearch.best_estimator_
'''
для рок аук
Лучшие значения параметров:
  C = 0.774263682681127,
  class_weight = balanced,
  dual = False,
  fit_intercept = True,
  intercept_scaling = 1,
  l1_ratio = None,
  max_iter = 1000,
  multi_class = auto,
  n_jobs = None,
  penalty = l2,
  random_state = 0,
  solver = lbfgs,
  tol = 0.001,
  verbose = 0,
  warm_start = False,
  Для f1
  Лучшие значения параметров:
  C = 16.68100537200059,
  class_weight = balanced,
  dual = False,
  fit_intercept = True,
  intercept_scaling = 1,
  l1_ratio = None,
  max_iter = 1000,
  multi_class = auto,
  n_jobs = None,
  penalty = l2,
  random_state = 0,
  solver = lbfgs,
  tol = 0.001,
  verbose = 0,
  warm_start = False,
'''
# Печатаем параметры
best_params = model_best_params.get_params()
print(f'Лучшие значения параметров:') 
for param_name in best_params.keys():
        print(f'  {param_name} = {best_params[param_name]},')

0.38788131367397843
[[64427     0]
 [ 9372     0]]
Fitting 5 folds for each of 240 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored

In [ ]:


X = df[df.label=='train'][['age',
       'decline_app_cnt', 'good_work', 'score_bki', 'bki_request_cnt',
       'region_rating', 'income',
       'first_time',
       'education_ACD', 'education_GRD', 'education_PGR', 'education_SCH',
       'education_UGR', 'sex_F', 'sex_M', 'car_type_N', 'car_type_Y',
       'foreign_passport_N', 'foreign_passport_Y']]

y = df[df.label=='train'].default

model = LogisticRegression(**best_params)
model.fit(X,y)
proba = model.predict_proba(X)
print(roc_auc_score(y,proba[:,0]))
X = df[df.label=='test'][['age',
       'decline_app_cnt', 'good_work', 'score_bki', 'bki_request_cnt',
       'region_rating', 'income',
       'first_time',
       'education_ACD', 'education_GRD', 'education_PGR', 'education_SCH',
       'education_UGR', 'sex_F', 'sex_M', 'car_type_N', 'car_type_Y',
       'foreign_passport_N', 'foreign_passport_Y']]
ids = df[df.label=='test'].client_id.reset_index()
proba = model.predict_proba(X)
proba = pd.DataFrame(proba[:,0])

ids['default'] = proba
ids.drop(['index'],axis=1,inplace=True)
ids.to_csv(data_path+'submit2.csv',index=False)


0.29052202732885557


In [ ]:
ids.default = 1-ids.default
ids.to_csv(data_path+'submit2.csv',index=False)

In [ ]:
roc_auc = roc_auc_score(y,proba)
print(roc_auc)

ValueError: ignored

AttributeError: ignored

In [ ]:
len(res)

36349

In [ ]:
res.to_csv(data_path+'res2.csv')

In [ ]:
t=pd.read_csv(data_path+'sample_submission.csv')
len(t)

24354